# **European patents BERTopic**

In [1]:
import torch

c:\Users\user\anaconda3\envs\bertopic_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0)) 

Is CUDA available: True
CUDA device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 4070 Laptop GPU


In [3]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.metrics import silhouette_score
import re

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

In [5]:
import os
os.getcwd()

'c:\\Users\\user\\OneDrive - TUM\\1. 연구\\In Progress\\Sci-tech semantic proximity and regional competitiveness\\Data\\Queries\\Sci-Tech Similarity'

In [6]:
import pandas as pd
patent_app = pd.read_csv('patent_app.csv')
patent_app

,appln_id,appln_kind,appln_filing_year,docdb_family_id,docdb_family_size
0,1,A,2000,8554171,6
1,2,A,1992,27517085,8
2,7,A,2000,10861834,7
3,9,A,2000,8555798,23
4,10,A,2001,7655749,9
...,...,...,...,...,...
8290059,606446150,U,2020,78048989,2
8290060,606446152,U,2022,89853283,1
8290061,606471480,A,2022,89854709,1
8290062,606471485,A,2022,89854710,1


In [7]:
# check number of unique appln_id 
patent_app['appln_id'].nunique()

8290064

In [8]:
patent_text = pd.read_csv('patent_text.csv')
patent_text

,appln_id,appln_title,appln_title_lg,appln_abstract,appln_abstract_lg
0,1,Method and means for using additional cards in...,en,A mobile station (MS) that comprises an interf...,en
1,2,Production of anti-self antibodies from antibo...,en,Methods are disclosed for the production of an...,en
2,7,MULTILAYER TELECOMMUNICATIONS NETWORK,en,A communications system comprising: a first ne...,en
3,9,SPRAYHEAD WITH NOZZLES MADE BY BORING,en,The invention relates to a spray head for prod...,en
4,10,Diphosphines,en,New enantiomericlly pure (substituted biphenyl...,en
...,...,...,...,...,...
8277625,606446150,Vorrichtung zur spektroskopischen Untersuchung...,de,Eine Vorrichtung (20) zur spektroskopischen Un...,de
8277626,606446152,Agro-photovoltaische Anlage,de,Die Erfindung betrifft eine Photovoltaikanlage...,de
8277627,606471480,SURFACE MODIFIED DOUGH CELLS AS BIORECOGNITION...,en,The present invenlion refers to the developmen...,en
8277628,606471485,MOBILE PHONE PROTECTIVE CASE/COVER WITH BIO-ME...,en,The invention relates to a cell phone case des...,en


In [9]:
patent_text['appln_id'].nunique()

8277630

In [10]:
df = pd.merge(patent_app, patent_text, on = 'appln_id', how = 'left')
df

,appln_id,appln_kind,appln_filing_year,docdb_family_id,docdb_family_size,appln_title,appln_title_lg,appln_abstract,appln_abstract_lg
0,1,A,2000,8554171,6,Method and means for using additional cards in...,en,A mobile station (MS) that comprises an interf...,en
1,2,A,1992,27517085,8,Production of anti-self antibodies from antibo...,en,Methods are disclosed for the production of an...,en
2,7,A,2000,10861834,7,MULTILAYER TELECOMMUNICATIONS NETWORK,en,A communications system comprising: a first ne...,en
3,9,A,2000,8555798,23,SPRAYHEAD WITH NOZZLES MADE BY BORING,en,The invention relates to a spray head for prod...,en
4,10,A,2001,7655749,9,Diphosphines,en,New enantiomericlly pure (substituted biphenyl...,en
...,...,...,...,...,...,...,...,...,...
8290059,606446150,U,2020,78048989,2,Vorrichtung zur spektroskopischen Untersuchung...,de,Eine Vorrichtung (20) zur spektroskopischen Un...,de
8290060,606446152,U,2022,89853283,1,Agro-photovoltaische Anlage,de,Die Erfindung betrifft eine Photovoltaikanlage...,de
8290061,606471480,A,2022,89854709,1,SURFACE MODIFIED DOUGH CELLS AS BIORECOGNITION...,en,The present invenlion refers to the developmen...,en
8290062,606471485,A,2022,89854710,1,MOBILE PHONE PROTECTIVE CASE/COVER WITH BIO-ME...,en,The invention relates to a cell phone case des...,en


In [11]:
df.isnull().sum()

appln_id                   0
appln_kind                 0
appln_filing_year          0
docdb_family_id            0
docdb_family_size          0
appln_title            12440
appln_title_lg         12434
appln_abstract       2110755
appln_abstract_lg    2110643
dtype: int64

In [12]:
# drop rows where appln_abstract is null 
df = df.dropna(subset=['appln_abstract'])
df.head()

,appln_id,appln_kind,appln_filing_year,docdb_family_id,docdb_family_size,appln_title,appln_title_lg,appln_abstract,appln_abstract_lg
0,1,A,2000,8554171,6,Method and means for using additional cards in...,en,A mobile station (MS) that comprises an interf...,en
1,2,A,1992,27517085,8,Production of anti-self antibodies from antibo...,en,Methods are disclosed for the production of an...,en
2,7,A,2000,10861834,7,MULTILAYER TELECOMMUNICATIONS NETWORK,en,A communications system comprising: a first ne...,en
3,9,A,2000,8555798,23,SPRAYHEAD WITH NOZZLES MADE BY BORING,en,The invention relates to a spray head for prod...,en
4,10,A,2001,7655749,9,Diphosphines,en,New enantiomericlly pure (substituted biphenyl...,en


In [13]:
# drop rows where appln_abstract_lg is not en 
df = df[df['appln_abstract_lg'] == 'en']
df.head()

,appln_id,appln_kind,appln_filing_year,docdb_family_id,docdb_family_size,appln_title,appln_title_lg,appln_abstract,appln_abstract_lg
0,1,A,2000,8554171,6,Method and means for using additional cards in...,en,A mobile station (MS) that comprises an interf...,en
1,2,A,1992,27517085,8,Production of anti-self antibodies from antibo...,en,Methods are disclosed for the production of an...,en
2,7,A,2000,10861834,7,MULTILAYER TELECOMMUNICATIONS NETWORK,en,A communications system comprising: a first ne...,en
3,9,A,2000,8555798,23,SPRAYHEAD WITH NOZZLES MADE BY BORING,en,The invention relates to a spray head for prod...,en
4,10,A,2001,7655749,9,Diphosphines,en,New enantiomericlly pure (substituted biphenyl...,en


In [14]:
df['appln_filing_year'].min(), df['appln_filing_year'].max()

(1980, 2022)

In [15]:
df.groupby('appln_filing_year')['appln_id'].nunique()

appln_filing_year
1980     34487
1981     34838
1982     35876
1983     34300
1984     39361
1985     44484
1986     49944
1987     50965
1988     55706
1989     58442
1990     59045
1991     57566
1992     67065
1993     83885
1994     97439
1995    107899
1996    110874
1997    124171
1998    132924
1999    141132
2000    147430
2001    150273
2002    158025
2003    173778
2004    175546
2005    184895
2006    174697
2007    167786
2008    167031
2009    155734
2010    164375
2011    167938
2012    175866
2013    169332
2014    159819
2015    162844
2016    156993
2017    149258
2018    137775
2019    114770
2020     86186
2021     48177
2022     16741
Name: appln_id, dtype: int64

**특허수가 감소함 -> 확인필요**

In [16]:
# create 'period' column 
def assign_period(appln_filing_year):
    if 1981 <= appln_filing_year <= 1985:
        return 1
    elif 1986 <= appln_filing_year <= 1990:
        return 2 
    elif 1991 <= appln_filing_year <= 1995:
        return 3 
    elif 1996 <= appln_filing_year <= 2000:
        return 4
    elif 2001 <= appln_filing_year <= 2005:
        return 5
    elif 2006 <= appln_filing_year <= 2010:
        return 6
    elif 2011 <= appln_filing_year <= 2015:
        return 7
    elif 2016 <= appln_filing_year <= 2020:
        return 8
    else:
        return 0 
    
df['period'] = df['appln_filing_year'].apply(assign_period)
df.head()

,appln_id,appln_kind,appln_filing_year,docdb_family_id,docdb_family_size,appln_title,appln_title_lg,appln_abstract,appln_abstract_lg,period
0,1,A,2000,8554171,6,Method and means for using additional cards in...,en,A mobile station (MS) that comprises an interf...,en,4
1,2,A,1992,27517085,8,Production of anti-self antibodies from antibo...,en,Methods are disclosed for the production of an...,en,3
2,7,A,2000,10861834,7,MULTILAYER TELECOMMUNICATIONS NETWORK,en,A communications system comprising: a first ne...,en,4
3,9,A,2000,8555798,23,SPRAYHEAD WITH NOZZLES MADE BY BORING,en,The invention relates to a spray head for prod...,en,4
4,10,A,2001,7655749,9,Diphosphines,en,New enantiomericlly pure (substituted biphenyl...,en,5


**NUTS 정보 추가**

In [17]:
patent_pers_1 = pd.read_csv('patent_pers_1.csv')
patent_pers_2 = pd.read_csv('patent_pers_2.csv')
patent_pers_3 = pd.read_csv('patent_pers_3.csv')

patent_pers = pd.concat([patent_pers_1, patent_pers_2, patent_pers_3], ignore_index=True)
patent_pers

,appln_id,person_id,psn_name,person_ctry_code,nuts,nuts_level,psn_sector,han_id,han_name,han_harmonized
0,1,1,NOKIA CORPORATION,FI,FI1B1,3,COMPANY,2125445,NOKIA CORP,2
1,1,2,"LIPPONEN, MARKKU",FI,FI197,3,NaN,100000002,"Lipponen, Markku",0
2,1,3,"LAITINEN, TIMO",FI,FI197,3,NaN,100000003,"Laitinen, Timo",0
3,1,4,"AHO, ARI",FI,FI197,3,NaN,100000004,"Aho, Ari",0
4,1,5,"KNUUTILA, JARNO",FI,FI197,3,NaN,100000005,"Knuutila, Jarno",0
...,...,...,...,...,...,...,...,...,...,...
27949106,606471485,91722704,"GIAKOUMIS, IOANNIS DIMITRIOU",GR,EL,0,UNKNOWN,191722704,"GIAKOUMIS, IOANNIS DIMITRIOU",0
27949107,606471485,91802500,"PANAGOPOULOU-PAPAIOANNOU, NIKI VASILEIOU",GR,EL,0,UNKNOWN,191802500,"PANAGOPOULOU-PAPAIOANNOU, NIKI VASILEIOU",0
27949108,606471485,92387371,"STAMATI, MARIA DIMITRIOU",GR,EL,0,UNKNOWN,192387371,"STAMATI, MARIA DIMITRIOU",0
27949109,606471485,92435692,"ARGYRI, SPYRIDOULA ILIA",GR,EL,0,UNKNOWN,192435692,"ARGYRI, SPYRIDOULA ILIA",0


In [18]:
# drop rows where nuts_level is 0 
patent_pers = patent_pers[patent_pers['nuts_level'] != 0]
patent_pers 

,appln_id,person_id,psn_name,person_ctry_code,nuts,nuts_level,psn_sector,han_id,han_name,han_harmonized
0,1,1,NOKIA CORPORATION,FI,FI1B1,3,COMPANY,2125445,NOKIA CORP,2
1,1,2,"LIPPONEN, MARKKU",FI,FI197,3,NaN,100000002,"Lipponen, Markku",0
2,1,3,"LAITINEN, TIMO",FI,FI197,3,NaN,100000003,"Laitinen, Timo",0
3,1,4,"AHO, ARI",FI,FI197,3,NaN,100000004,"Aho, Ari",0
4,1,5,"KNUUTILA, JARNO",FI,FI197,3,NaN,100000005,"Knuutila, Jarno",0
...,...,...,...,...,...,...,...,...,...,...
27949022,606406381,79724695,"HADAP, SUNIL SHARADCHANDRA",US,NaN,9,NaN,179724695,"Hadap, Sunil Sharadchandra",0
27949023,606406381,7306804,AMAZON TECHNOLOGIES,US,NaN,9,COMPANY,100043,AMAZON TECH INC,2
27949026,606406383,47257652,GO PRO,US,NaN,9,COMPANY,1374307,GOPRO INC,2
27949028,606406445,77863828,"SCHONER, BERND",US,NaN,9,NaN,177863828,"Schoner, Bernd",0


In [19]:
nuts = patent_pers[['appln_id', 'nuts', 'nuts_level']].drop_duplicates()
nuts

,appln_id,nuts,nuts_level
0,1,FI1B1,3
1,1,FI197,3
5,2,UKI32,3
6,2,UKH12,3
9,2,NaN,9
...,...,...,...
27949017,606406375,NaN,9
27949021,606406381,NaN,9
27949026,606406383,NaN,9
27949028,606406445,NaN,9


In [20]:
# drop NaN 
nuts = nuts.dropna(subset=['nuts'])
nuts

,appln_id,nuts,nuts_level
0,1,FI1B1,3
1,1,FI197,3
5,2,UKI32,3
6,2,UKH12,3
13,7,FI1B1,3
...,...,...,...
27942476,604927030,SE232,4
27943200,605163630,UKJ23,4
27943526,605222772,DE21J,4
27948230,606280441,CH032,4


In [21]:
# check number of data for each unique value in nuts_level 
nuts['nuts_level'].value_counts()

nuts_level
3    2244647
4     313123
1        916
2        314
Name: count, dtype: int64

In [22]:
# remove rows where nuts_level is 1 or 2 
nuts = nuts[~nuts['nuts_level'].isin([1, 2])]
nuts

,appln_id,nuts,nuts_level
0,1,FI1B1,3
1,1,FI197,3
5,2,UKI32,3
6,2,UKH12,3
13,7,FI1B1,3
...,...,...,...
27942476,604927030,SE232,4
27943200,605163630,UKJ23,4
27943526,605222772,DE21J,4
27948230,606280441,CH032,4


In [23]:
# drop nuts_level 
nuts = nuts.drop(columns=['nuts_level'])
nuts.head()

,appln_id,nuts
0,1,FI1B1
1,1,FI197
5,2,UKI32
6,2,UKH12
13,7,FI1B1


In [24]:
# drop rows where nuts = AN000 
nuts = nuts[nuts['nuts'] != 'AN000']
nuts.head()

,appln_id,nuts
0,1,FI1B1
1,1,FI197
5,2,UKI32
6,2,UKH12
13,7,FI1B1


In [25]:
# slice nuts to 4 digits 
nuts['nuts'] = nuts['nuts'].str.slice(0, 4)
nuts.head()

,appln_id,nuts
0,1,FI1B
1,1,FI19
5,2,UKI3
6,2,UKH1
13,7,FI1B


# **특정 기준 이상 특허 보유한 지역, 기간 선정**

In [26]:
df = pd.merge(df, nuts, on = 'appln_id', how = 'left')
df.head()

,appln_id,appln_kind,appln_filing_year,docdb_family_id,docdb_family_size,appln_title,appln_title_lg,appln_abstract,appln_abstract_lg,period,nuts
0,1,A,2000,8554171,6,Method and means for using additional cards in...,en,A mobile station (MS) that comprises an interf...,en,4,FI1B
1,1,A,2000,8554171,6,Method and means for using additional cards in...,en,A mobile station (MS) that comprises an interf...,en,4,FI19
2,2,A,1992,27517085,8,Production of anti-self antibodies from antibo...,en,Methods are disclosed for the production of an...,en,3,UKI3
3,2,A,1992,27517085,8,Production of anti-self antibodies from antibo...,en,Methods are disclosed for the production of an...,en,3,UKH1
4,7,A,2000,10861834,7,MULTILAYER TELECOMMUNICATIONS NETWORK,en,A communications system comprising: a first ne...,en,4,FI1B


In [27]:
# group by nuts and period and count the number of unique appln_id 
df_grouped = df.groupby(['nuts', 'period'])['appln_id'].nunique().reset_index()
df_grouped.head()

,nuts,period,appln_id
0,AT11,0,3
1,AT11,1,11
2,AT11,2,14
3,AT11,3,26
4,AT11,4,66


In [28]:
df_grouped.describe()

,period,appln_id
count,2631.000000,2631.000000
mean,4.219688,660.625998
std,2.567049,1605.544340
min,0.000000,1.000000
25%,2.000000,20.000000
50%,4.000000,142.000000
75%,6.000000,595.500000
max,8.000000,21250.000000


In [29]:
# drop rows where pubid is less than 50 
df_grouped = df_grouped[df_grouped['appln_id'] >= 100]
df_grouped

,nuts,period,appln_id
5,AT11,5,116
6,AT11,6,167
7,AT11,7,176
10,AT12,1,129
11,AT12,2,300
...,...,...,...
2626,UKN0,4,118
2627,UKN0,5,155
2628,UKN0,6,236
2629,UKN0,7,384


In [30]:
# check number of unique values in nuts 
df_grouped['nuts'].nunique()

223

In [31]:
df_grouped.to_csv('patent_above_100.csv', index=False)

In [32]:
import pandas as pd 
patent = pd.read_csv('patent_above_100.csv')
pub = pd.read_csv('pub_above_100.csv')

In [33]:
df_grouped = pd.merge(df_grouped, pub, left_on=['nuts', 'period'], right_on=['eu_nuts_id', 'period'], how = 'inner')

# drop pubid columns 
df_grouped = df_grouped.drop(columns=['pubid'])
df_grouped.head()

,nuts,period,appln_id,eu_nuts_id
0,AT12,4,774,AT12
1,AT12,5,994,AT12
2,AT12,6,1283,AT12
3,AT13,4,1134,AT13
4,AT13,5,1662,AT13


In [34]:
# drop rows in df where each pair of nuts and period is not in df_grouped 
df = df.merge(df_grouped[['nuts', 'period']], on=['nuts', 'period'], how='inner')
df.head()

,appln_id,appln_kind,appln_filing_year,docdb_family_id,docdb_family_size,appln_title,appln_title_lg,appln_abstract,appln_abstract_lg,period,nuts
0,1,A,2000,8554171,6,Method and means for using additional cards in...,en,A mobile station (MS) that comprises an interf...,en,4,FI1B
1,7,A,2000,10861834,7,MULTILAYER TELECOMMUNICATIONS NETWORK,en,A communications system comprising: a first ne...,en,4,FI1B
2,9,A,2000,8555798,23,SPRAYHEAD WITH NOZZLES MADE BY BORING,en,The invention relates to a spray head for prod...,en,4,FI1B
3,775,A,1999,8552589,7,COMPRESSION OF NODES IN A TRIE STRUCTURE,en,The invention relates to a method for implemen...,en,4,FI1B
4,1187,A,2000,8555622,4,Charging for telecommunications services,en,"A method for prepayment of a service, comprisi...",en,4,FI1B


In [35]:
# customize stop words list
custom_stop_words = ["determining", "method", "includes", "based", "second", "using", 'showed', 'problem', "study", "non"
                     "12", "14", "15", "2014", "iii", "ii", "10", "1a", "11", "cm", "60", '28', '250', '17',
                     "25", "40", "model", "2005", "ar", "rh", "kv", "sb", "ac", "www", "nm", 'srr', 'xy', 'sigma',
                     "mo", "nm", "18", "ee", "pb", "gev", "mu", "rh", "ln", "a15", "13", "ag", '55', 'effect','nd3',
                     "nm", "qds", "iv", "ag", "rms", "le", "p53", "pl", "fe", "ir", "4abn", "nmr", "ni", '05', '74',
                     "ta", "tio2"< "thz", "db", "hz", "iso", "pss", "1310", "nu", "tf", "tf", '33', '350',
                     "km", "tev", "gev", "fb", "tl", "t2dm", "nlc", "dot", "si", "cb7", "mc", "soi", '100', 
                     "dots", "pi", "qd", "la", "cdw", "iqp", "ccl4", "no2", "mm", "p1", "bi", "ho", "rs", '00',
                     "qh", "tio2", "nu", "nir", "time", "bm12", "ssi", "u6", "current", "nmr", 'hs', 'sp',
                     "gev", "tot", "1h", "hh", "hi", "qds", "pss", "no2", "zb", "wz", "al", "ge", 'ff', 'der',
                     "ect", "mdd", "95", "girls", "uc", "cpe", "di", "uc", "er3", "new", 's190', 's208', '44',
                     "od", "ch3", "yb", "eu", "tb", "nr", "ope3", "4tp", "mu", "center", "la", '3b', 'eag1',
                     "number", "su", "ch4", "fe", "cr", "ni", "eat", "ws", "spp", "mn2", "mm", 'ss', 'oc',
                     "rf", "li", "2016", "16", "sic", "si", "von", "sept3", "0d", "36", 'igf1r', 'ma', 'mnc', ''
                     'o3', 'correctly', 'tb', 'te', 'il', 'cd39', 'cd73', 'au', 'ou', 'ml', 'mg', 'puo2', '298',
                     'sec', 'eta', 'ba122', 'al', 'cgm', 'h2s', 'ir', 'sni', '45', '75', 'tio2', '60', 'sp', '4000', 
                     'ge', 'si', '5p', '120', '2d', 'random', 'theory', 'li', 'qds', 'qd', 'p3ht', 'mu', '69', '71',
                     'p2', 'gw', '511', '3885', 'yb3', '25', 'sir', '240', 'ro', 't34', '10', '6p', 'bbb', 'c1', 'c3',
                     'nno', 'nlo', 'ag', 'sb', 'naf', 'ga', 'se', 'cu', 'gd', 'pf6', 'dots', 'dat', '4f', 
                     'ch3', 'center', 'centre', 'mc3t3', 'e1', '1064', 'jc', 'ga', 'cf3', 'bu', 'zn', 'near', 
                     'np', 'zn', 'cp', 'fr', 'nmc', 'nca', 'oco', 'dsb', 'ssb', 'oe', 'mn', '300', '80', 'b12x12',
                     'nh', 'br', 'mw', '77', 'ig', '24', 'sigma', '700', 'ij', '1h', 'approach', '4b', '343', 
                     's6', 'ce3', 'advantages', 'phase', '001', 'ad', 'dy3', '111', '2000', 'matter', '99',
                     'results', 'used', 'methods', 'analysis', 'used', 'associated', 'different', 'argue', 'article', 
                     'let', 'research', 'data', 'paper', 'examines', 'prove', 'proposed', 'samples', 'study', 'similar', 
                     'compared', 'significantly', 'high', 'publication', 'significant', 'use', 'studies', 'category', 
                     'use', 'field', 'group', 'years', '000', '2024', 'field', 'risk', 'non', 'age'
                     ]  

# 기본 영어 stop words (서술어, 조사 등) 목록 가져오기
default_stop_words = text.ENGLISH_STOP_WORDS

# 기본 영어 stop words에 추가할 단어들 합치기
all_stop_words = list(default_stop_words.union(custom_stop_words))

**Hyperparameter tuning**

In [ ]:
import pandas as pd
from tqdm import tqdm
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from bertopic.vectorizers import ClassTfidfTransformer
from itertools import product

# Create an empty list to store all results
all_results = []

# Define hyperparameter grid
n_gram_options = [(1, 1), (1, 2)]
min_dist_options = [0.0, 0.1, 0.3]
nr_topics_options = [10, 20, None]
min_topic_ratios = [0.01, 0.02]  # Percent of total docs for min_topic_size

# Create all hyperparameter combinations
hyperparams = list(product(n_gram_options, min_dist_options, nr_topics_options, min_topic_ratios))

# Initialize embedding model once
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Loop over each NUTS region
for nuts in tqdm(df['nuts'].unique(), desc="NUTS regions"):
    for i in range(1, 9):
        filtered_df = df[(df['nuts'] == nuts) & (df['period'] == i)].copy()
        docs = filtered_df['appln_abstract'].dropna().astype(str).tolist()

        if len(docs) < 5:
            print(f"Skipping NUTS {nuts}, period {i} due to insufficient documents.")
            continue

        embeddings = embedding_model.encode(docs, show_progress_bar=True)

        for n_gram, min_dist, nr_topics, min_ratio in hyperparams:
            try:
                min_topic_size = max(2, int(len(docs) * min_ratio))
                n_neighbors = min(15, len(docs) - 1)
                if n_neighbors < 2:
                    continue

                vectorizer_model = CountVectorizer(stop_words=all_stop_words)
                ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
                umap_model = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, metric='cosine')

                topic_model = BERTopic(
                    n_gram_range=n_gram,
                    min_topic_size=min_topic_size,
                    nr_topics=nr_topics,
                    embedding_model=embedding_model,
                    vectorizer_model=vectorizer_model,
                    calculate_probabilities=True,
                    ctfidf_model=ctfidf_model,
                    umap_model=umap_model
                )

                topics, probs = topic_model.fit_transform(docs, embeddings)
                freq = topic_model.get_topic_info()

                # Add metadata
                freq['nuts'] = nuts
                freq['period'] = i
                freq['n_gram'] = str(n_gram)
                freq['min_dist'] = min_dist
                freq['nr_topics'] = nr_topics
                freq['min_topic_ratio'] = min_ratio

                all_results.append(freq)
                print(f"✓ NUTS {nuts}, period {i} - ngram: {n_gram}, min_dist: {min_dist}, topics: {nr_topics}, min_ratio: {min_ratio}")

            except Exception as e:
                print(f"✗ Error for NUTS {nuts}, period {i} - params: {n_gram}, {min_dist}, {nr_topics}, {min_ratio} -> {e}")

# Combine all results
final_results = pd.concat(all_results, ignore_index=True)
final_results.to_csv('patent_bertopic_hyperopt.csv', index=False)

**BERTopic**

In [56]:
import pandas as pd
from tqdm import tqdm
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from bertopic.vectorizers import ClassTfidfTransformer

# Create an empty list to store the results
all_results = []

# Iterate over each 'nuts' and 'period', showing progress with tqdm
for nuts in tqdm(df['nuts'].unique(), desc="NUTS regions"):
    for i in range(1, 9):
        # Create a filtered copy of the dataset for 'nuts' and 'period'
        filtered_df = df[(df['nuts'] == nuts) & (df['period'] == i)].copy()

        # Extract the 'appln_abstract' column for BERTopic modeling, and ensure all entries are strings
        docs = filtered_df['appln_abstract'].dropna().astype(str).tolist()

        if len(docs) == 0:
            print(f"No valid documents for NUTS region {nuts} and period {i}")
            continue

        try:
            # Term Frequency, Inverse Document Frequency & transform docs into vectors
            vectorizer_model = CountVectorizer(stop_words=all_stop_words)
            ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
            embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
            embeddings = embedding_model.encode(docs, show_progress_bar=True)

            # Dynamically set the number of neighbors for UMAP based on the number of documents
            n_neighbors = min(15, len(docs) - 1)  # Ensures n_neighbors <= number of documents 15 or 30
            if n_neighbors <= 2:  # UMAP requires n_neighbors to be at least 2
                print(f"Skipping NUTS region {nuts} and period {i} due to insufficient documents.")
                continue

            umap_model = UMAP(n_neighbors=n_neighbors, min_dist=0.1, metric='cosine') # 0.0-0.3, a lower min_dist makes the clusters tighgter, whereas a higher value makes dispersed clusters

            # hyper parameter 설정, 최대 10개의 주제를, 최소 min_topic_sizes 보다 큰 클러스터로 생성 
            n_gram_ranges = (1, 2)  # n-gram 범위 (1, 1), (1, 2), (2, 2)
            min_topic_sizes = max(2, int(len(docs) * 0.03))  # 한 클러스터에 포함되는 최소 문서 수 (1, 1.5, 2, 3%)
            nr_topics_options = 20  # 최대 클러스터링 수 (5, 10, 15, 20개)

            # 문서별로 할당된 주제 topics와 확률 probs
            topic_model = BERTopic(
                n_gram_range=n_gram_ranges,
                min_topic_size=min_topic_sizes,
                nr_topics=nr_topics_options,
                embedding_model=embedding_model,
                vectorizer_model=vectorizer_model,
                calculate_probabilities=True,
                ctfidf_model=ctfidf_model,
                umap_model=umap_model
            )

            topics, probs = topic_model.fit_transform(docs, embeddings)

            # 결과를 저장
            freq = topic_model.get_topic_info()

            # Add 'nuts' and 'period' columns to the topic info DataFrame
            freq['nuts'] = nuts
            freq['period'] = i

            # Append the result to the list
            all_results.append(freq)

            print(f"Processed topic information for NUTS region {nuts} and period {i}")

        except Exception as e:
            print(f"Error processing NUTS region {nuts} and period {i}: {e}")

# Combine all the results into a single DataFrame
final_results = pd.concat(all_results, ignore_index=True)

# Save the final combined DataFrame to a single CSV file
final_results.to_csv('patent_bertopic.csv', index=False)

NUTS regions:   0%|          | 0/203 [00:00<?, ?it/s]

No valid documents for NUTS region FI1B and period 1
No valid documents for NUTS region FI1B and period 2
No valid documents for NUTS region FI1B and period 3


Batches: 100%|██████████| 98/98 [00:14<00:00,  6.92it/s]


Processed topic information for NUTS region FI1B and period 4


Batches: 100%|██████████| 126/126 [00:18<00:00,  6.78it/s]


...

Processed topic information for NUTS region ES11 and period 7
No valid documents for NUTS region ES11 and period 8
No valid documents for NUTS region PL41 and period 1
No valid documents for NUTS region PL41 and period 2
No valid documents for NUTS region PL41 and period 3
No valid documents for NUTS region PL41 and period 4
No valid documents for NUTS region PL41 and period 5
No valid documents for NUTS region PL41 and period 6


NUTS regions: 100%|█████████▉| 202/203 [3:58:40<00:21, 21.19s/it]

Processed topic information for NUTS region PL41 and period 7
No valid documents for NUTS region PL41 and period 8
No valid documents for NUTS region PT16 and period 1
No valid documents for NUTS region PT16 and period 2
No valid documents for NUTS region PT16 and period 3
No valid documents for NUTS region PT16 and period 4
No valid documents for NUTS region PT16 and period 5
No valid documents for NUTS region PT16 and period 6


NUTS regions: 100%|██████████| 203/203 [3:58:58<00:00, 70.63s/it]

Processed topic information for NUTS region PT16 and period 7
No valid documents for NUTS region PT16 and period 8


In [2]:
import pandas as pd
final_results = pd.read_csv('eu_patent_bertopic_nuts2.csv')
final_results.head()

,Topic,Count,Name,Representation,Representative_Docs,nuts,period
0,-1,389,-1_speech_image_signal_imaging,"['speech', 'image', 'signal', 'imaging', 'ante...","[""A module (1') adapted for insertion into a d...",FI1B,4
1,0,1435,0_web_invention_material_roll,"['web', 'invention', 'material', 'roll', 'rela...",['A twin-wire gap former in a paper machine in...,FI1B,4
2,1,100,1_display_housing_user_keys,"['display', 'housing', 'user', 'keys', 'handse...",['A radiophone is provided with a user interfa...,FI1B,4
3,2,1191,2_network_mobile_station_radio,"['network', 'mobile', 'station', 'radio', 'inf...",['The present invention proposes a method for ...,FI1B,4
4,-1,688,-1_elevator_hoisting_rope_ropes,"['elevator', 'hoisting', 'rope', 'ropes', 'car...",['The invention relates to a method for modern...,FI1B,5


In [3]:
final_results[final_results['Topic'] == -1]

,Topic,Count,Name,Representation,Representative_Docs,nuts,period
0,-1,389,-1_speech_image_signal_imaging,"['speech', 'image', 'signal', 'imaging', 'ante...","[""A module (1') adapted for insertion into a d...",FI1B,4
4,-1,688,-1_elevator_hoisting_rope_ropes,"['elevator', 'hoisting', 'rope', 'ropes', 'car...",['The invention relates to a method for modern...,FI1B,5
9,-1,538,-1_elevator_said_antenna_comprises,"['elevator', 'said', 'antenna', 'comprises', '...",['A multi-layer dental panoramic and transvers...,FI1B,6
13,-1,506,-1_web_treatment_imaging_winding,"['web', 'treatment', 'imaging', 'winding', 'po...",['The invention relates to a medical computed ...,FI1B,7
19,-1,160,-1_charging_glass_video_switching,"['charging', 'glass', 'video', 'switching', 's...",['The invention discloses a method of improvin...,FI19,5
...,...,...,...,...,...,...,...
5050,-1,19,-1_sample_modified_rectangular_mastitis,"['sample', 'modified', 'rectangular', 'mastiti...",['The invention is provided for detection of H...,EE00,6
5070,-1,10,-1_pile_nucleotide_sleeve_interlock,"['pile', 'nucleotide', 'sleeve', 'interlock', ...",['A vent (10) for a mold having a mold cavity ...,BE34,5
5083,-1,28,-1_cache_tags_visor_acid,"['cache', 'tags', 'visor', 'acid', 'ceramic', ...","['Process for enriching microalgal biomass, in...",ES11,7
5099,-1,21,-1_test_elevator_car_photoluminescent,"['test', 'elevator', 'car', 'photoluminescent'...",['The invention relates to a method for develo...,PL41,7


In [4]:
# Step 1: Total count per nuts and period
total_count = final_results.groupby(['nuts', 'period'])['Count'].sum()

# Step 2: Count for Topic == -1 per nuts and period
minus_one_count = final_results[final_results['Topic'] == -1].groupby(['nuts', 'period'])['Count'].sum()

# Step 3: Calculate ratio
ratio = (minus_one_count / total_count).reset_index(name='topic_minus_one_ratio')

# Optional: rename columns for clarity
ratio = ratio.rename(columns={'nuts': 'NUTS', 'period': 'Period'})
ratio

,NUTS,Period,topic_minus_one_ratio
0,AT12,4,0.158659
1,AT12,5,0.043441
2,AT12,6,0.057731
3,AT13,4,0.261905
4,AT13,5,0.037215
...,...,...,...
664,UKM5,6,0.004598
665,UKN0,4,0.130435
666,UKN0,5,0.196078
667,UKN0,6,0.175385


In [5]:
ratio['topic_minus_one_ratio'].describe()

count    605.000000
mean       0.175335
std        0.121484
min        0.001229
25%        0.075342
50%        0.165217
75%        0.256659
max        0.604601
Name: topic_minus_one_ratio, dtype: float64

# **ChatGPT를 활용한 Label 생성**

In [ ]:
from openai import OpenAI

# client = OpenAI(api_key="")

final_results = pd.read_csv('eu_patent_bertopic_nuts2.csv')

# List to hold results 
results = []

# Iterate through each row in the DataFrame
for index, row in final_results.iterrows():
    keywords = row['Representation']

    # Prompt for OpenAI
    prompt = f'''{keywords}
    The list of words are the outcome of the topic modeling from patent publications.
    Generate a new label that best describes the topic of the articles as follow: <new science topic>
    Provide a specific and detailed label. I want just a single label. I do not want descriptions for the labels.'''

    try:
        # Call the OpenAI API to generate the new label
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an AI assistant that generates labels for topics based on provided keywords."},
                {"role": "user", "content": prompt}
            ]
        )

        # Get the label from the response 
        new_label = response.choices[0].message.content.strip()

        # Append the result to the list
        results.append({
            'nuts' : row['nuts'],
            'period' : row['period'],
            'keyword' : keywords,
            'content' : new_label
        })
    
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv('eu_patent_bertopic_nuts2_labels.csv', index=False)